In [1]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt
import backtrader.indicators as btind

from backtrader.feeds import GenericCSVData

from statistics import mean
from statistics import median

In [2]:
class custom_csv(GenericCSVData):
    
    lines= ('UPDAY',)
    
    params = (
        ('dtformat', '%m/%d/%Y'),
        ('datetime', 0),
        ('time', -1),
        ('open', 1),
        ('high', 2),
        ('low', 3),
        ('close', 4),
        ('volume', 5),
        ('openinterest', -1),
        ('UPDAY', 6),
    )

In [3]:
#Optimize Trend Strategy
class Trendopt(bt.Strategy):
    params = (
        ('printlog', False),
        ('doprint', False),
        ('updaymaperiod', 39),
        ('buythresh', 59),
        ('sellthresh', 43),
    )
    
    def start(self):
        self.val_start = self.broker.get_cash()  # keep the starting cash
        self.daysoutofposition = 0

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        #Add UP DAY MA CALCS
        self.updayma = bt.indicators.MovingAverageSimple(
           self.data.UPDAY, period=self.params.updaymaperiod)
#         print(self.updayma)
        self.updayma.plotinfo.plotname = 'up day sma'

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        self.bar_executed = len(self)
        if not trade.isclosed:
            return
        self.bar_executed = len(self)

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
        #make a list of the pnl for each trade
        self.pnl_percent = trade.pnl / self.broker.getvalue()
        self.pnl_list=[]
        self.pnl_list.append(self.pnl_percent)
        
    def nextstart(self):
        # Buy all the available cash
        size = int(self.broker.get_cash() / self.data)
        self.buy(size=size)        
        

    def next(self):
        self.bar_executed = len(self)
        
#         self.daysoutofposition += 1
#         print(self.daysoutofposition)
        self.duration = len(self) - self.bar_executed + 1
        # Simply log the closing price of the series from the reference
        #self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return
        # Check if we are in the market
        if not self.position:
            self.daysoutofposition += 1
#             print(self.daysoutofposition)
        # Check if we are in the market
        if not self.position:

            # buy when updayma above threshold
            if self.updayma[0] >= (self.params.buythresh/100):
#                 print(self.params.buythresh/100)
#                 print(self.updayma[0])
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                size = int(self.broker.get_cash() / self.data)
                self.buy(size=size)   
                
        else:
            
            # sell when updayma below threshold
             if self.updayma[0] <= (self.params.sellthresh/100):
                 # SELL, SELL, SELL!!! (with all possible default parameters)
#                 print(self.params.sellthresh/100)
#                 print(self.updayma[0])
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.close()
                
    def stop(self):
        #Get final portfolio Value
        portvalue = cerebro.broker.getvalue()
        pnl = portvalue - startcash
        
        #roi
        self.roi = ((self.broker.get_value()-self.val_start) / self.val_start) * 100
        #take the mean if a pnl exists
        try:
            self.pnl_mean = (mean(self.pnl_list) * 100)
        except:
            self.pnl_list = [0]
            self.pnl_mean = (mean(self.pnl_list) * 100)

        #Print out the final result
        #self.log('Final Portfolio Value: $%.2f' % portvalue ,doprint=True)
        #self.log('P/L: $%.2f' % pnl, doprint=True)
        #self.log('P/L: %.2f%%' % (((portvalue - startcash)/startcash)*100), doprint=True)
        print('(UP DAY MA PERIOD %.0f) (BUY THRESH %.2f) (SELL THRESH %.2f) (MEAN TRADE PNL: %.2f%%) ROI %.2f%% Ending Value %.0f' %
        (self.params.updaymaperiod, self.params.buythresh/100, self.params.sellthresh/100, self.pnl_mean, self.roi, self.broker.getvalue()))
        


In [4]:
#runROC Opt Strategy
if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro(stdstats=False,optreturn=True,optdatas=True)
    cerebro.broker.set_coc(True)
    
    strats = cerebro.optstrategy(
        Trendopt,
        updaymaperiod=range(20, 51),
        buythresh=range(52,61),
        sellthresh=range(40,51)) 
#     cerebro.addstrategy(Trendopt)
    
    
    # Datas are in a subfolder of the samples. Need to find where the script is
    # because it could have been called from anywhere
    modpath = os.path.dirname(os.path.abspath(sys.argv[0]))
    datapath = os.path.join(modpath, 'C:\\Users\\big_u\\TREND WALK FORWARD TESTING\\UPDAYS2.csv')

    # Create a Data Feed
    data = custom_csv(
        dataname=datapath,
    )

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)
    startcash = 100000

    # Add a FixedSize sizer according to the stake
    cerebro.addsizer(bt.sizers.FixedSize, stake=1)

    # Print out the starting conditions
    #print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    
#   cerebro.addwriter(bt.WriterFile, csv = True, out='trend_results.csv')

    # Run over everything
    cerebro.run(maxcpus=1)
    


    
    # Set the commission
    cerebro.broker.setcommission(commission=0.00)

    #Print out the final result
    #print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())


(UP DAY MA PERIOD 20) (BUY THRESH 0.52) (SELL THRESH 0.40) (MEAN TRADE PNL: 3.29%) ROI 263.94% Ending Value 363938
(UP DAY MA PERIOD 20) (BUY THRESH 0.52) (SELL THRESH 0.41) (MEAN TRADE PNL: 3.29%) ROI 263.94% Ending Value 363938
(UP DAY MA PERIOD 20) (BUY THRESH 0.52) (SELL THRESH 0.42) (MEAN TRADE PNL: 3.29%) ROI 263.94% Ending Value 363938
(UP DAY MA PERIOD 20) (BUY THRESH 0.52) (SELL THRESH 0.43) (MEAN TRADE PNL: 3.29%) ROI 263.94% Ending Value 363938
(UP DAY MA PERIOD 20) (BUY THRESH 0.52) (SELL THRESH 0.44) (MEAN TRADE PNL: 3.29%) ROI 263.94% Ending Value 363938
(UP DAY MA PERIOD 20) (BUY THRESH 0.52) (SELL THRESH 0.45) (MEAN TRADE PNL: -1.88%) ROI 137.34% Ending Value 237338
(UP DAY MA PERIOD 20) (BUY THRESH 0.52) (SELL THRESH 0.46) (MEAN TRADE PNL: -1.88%) ROI 137.34% Ending Value 237338
(UP DAY MA PERIOD 20) (BUY THRESH 0.52) (SELL THRESH 0.47) (MEAN TRADE PNL: -1.88%) ROI 137.34% Ending Value 237338
(UP DAY MA PERIOD 20) (BUY THRESH 0.52) (SELL THRESH 0.48) (MEAN TRADE PNL: -

(UP DAY MA PERIOD 20) (BUY THRESH 0.58) (SELL THRESH 0.45) (MEAN TRADE PNL: -2.29%) ROI 143.87% Ending Value 243875
(UP DAY MA PERIOD 20) (BUY THRESH 0.58) (SELL THRESH 0.46) (MEAN TRADE PNL: -2.29%) ROI 143.87% Ending Value 243875
(UP DAY MA PERIOD 20) (BUY THRESH 0.58) (SELL THRESH 0.47) (MEAN TRADE PNL: -2.29%) ROI 143.87% Ending Value 243875
(UP DAY MA PERIOD 20) (BUY THRESH 0.58) (SELL THRESH 0.48) (MEAN TRADE PNL: -2.29%) ROI 143.87% Ending Value 243875
(UP DAY MA PERIOD 20) (BUY THRESH 0.58) (SELL THRESH 0.49) (MEAN TRADE PNL: -2.29%) ROI 143.87% Ending Value 243875
(UP DAY MA PERIOD 20) (BUY THRESH 0.58) (SELL THRESH 0.50) (MEAN TRADE PNL: 2.99%) ROI 207.73% Ending Value 307730
(UP DAY MA PERIOD 20) (BUY THRESH 0.59) (SELL THRESH 0.40) (MEAN TRADE PNL: 3.20%) ROI 241.18% Ending Value 341177
(UP DAY MA PERIOD 20) (BUY THRESH 0.59) (SELL THRESH 0.41) (MEAN TRADE PNL: 3.20%) ROI 241.18% Ending Value 341177
(UP DAY MA PERIOD 20) (BUY THRESH 0.59) (SELL THRESH 0.42) (MEAN TRADE PNL:

(UP DAY MA PERIOD 21) (BUY THRESH 0.56) (SELL THRESH 0.40) (MEAN TRADE PNL: 2.88%) ROI 229.87% Ending Value 329874
(UP DAY MA PERIOD 21) (BUY THRESH 0.56) (SELL THRESH 0.41) (MEAN TRADE PNL: 2.88%) ROI 229.87% Ending Value 329874
(UP DAY MA PERIOD 21) (BUY THRESH 0.56) (SELL THRESH 0.42) (MEAN TRADE PNL: 2.88%) ROI 229.87% Ending Value 329874
(UP DAY MA PERIOD 21) (BUY THRESH 0.56) (SELL THRESH 0.43) (MEAN TRADE PNL: 0.05%) ROI 270.45% Ending Value 370452
(UP DAY MA PERIOD 21) (BUY THRESH 0.56) (SELL THRESH 0.44) (MEAN TRADE PNL: 0.05%) ROI 270.45% Ending Value 370452
(UP DAY MA PERIOD 21) (BUY THRESH 0.56) (SELL THRESH 0.45) (MEAN TRADE PNL: 0.05%) ROI 270.45% Ending Value 370452
(UP DAY MA PERIOD 21) (BUY THRESH 0.56) (SELL THRESH 0.46) (MEAN TRADE PNL: 0.05%) ROI 270.45% Ending Value 370452
(UP DAY MA PERIOD 21) (BUY THRESH 0.56) (SELL THRESH 0.47) (MEAN TRADE PNL: 0.05%) ROI 270.45% Ending Value 370452
(UP DAY MA PERIOD 21) (BUY THRESH 0.56) (SELL THRESH 0.48) (MEAN TRADE PNL: -1.4

(UP DAY MA PERIOD 22) (BUY THRESH 0.53) (SELL THRESH 0.45) (MEAN TRADE PNL: 1.53%) ROI 314.01% Ending Value 414007
(UP DAY MA PERIOD 22) (BUY THRESH 0.53) (SELL THRESH 0.46) (MEAN TRADE PNL: -1.88%) ROI 222.63% Ending Value 322634
(UP DAY MA PERIOD 22) (BUY THRESH 0.53) (SELL THRESH 0.47) (MEAN TRADE PNL: -1.88%) ROI 222.63% Ending Value 322634
(UP DAY MA PERIOD 22) (BUY THRESH 0.53) (SELL THRESH 0.48) (MEAN TRADE PNL: -1.88%) ROI 222.63% Ending Value 322634
(UP DAY MA PERIOD 22) (BUY THRESH 0.53) (SELL THRESH 0.49) (MEAN TRADE PNL: -1.88%) ROI 222.63% Ending Value 322634
(UP DAY MA PERIOD 22) (BUY THRESH 0.53) (SELL THRESH 0.50) (MEAN TRADE PNL: -0.93%) ROI 191.97% Ending Value 291974
(UP DAY MA PERIOD 22) (BUY THRESH 0.54) (SELL THRESH 0.40) (MEAN TRADE PNL: -5.10%) ROI 611.28% Ending Value 711283
(UP DAY MA PERIOD 22) (BUY THRESH 0.54) (SELL THRESH 0.41) (MEAN TRADE PNL: 1.53%) ROI 314.01% Ending Value 414007
(UP DAY MA PERIOD 22) (BUY THRESH 0.54) (SELL THRESH 0.42) (MEAN TRADE PNL

(UP DAY MA PERIOD 22) (BUY THRESH 0.59) (SELL THRESH 0.50) (MEAN TRADE PNL: 2.82%) ROI 179.67% Ending Value 279672
(UP DAY MA PERIOD 22) (BUY THRESH 0.60) (SELL THRESH 0.40) (MEAN TRADE PNL: -8.19%) ROI 244.94% Ending Value 344937
(UP DAY MA PERIOD 22) (BUY THRESH 0.60) (SELL THRESH 0.41) (MEAN TRADE PNL: -0.12%) ROI 152.99% Ending Value 252989
(UP DAY MA PERIOD 22) (BUY THRESH 0.60) (SELL THRESH 0.42) (MEAN TRADE PNL: -0.12%) ROI 152.99% Ending Value 252989
(UP DAY MA PERIOD 22) (BUY THRESH 0.60) (SELL THRESH 0.43) (MEAN TRADE PNL: -0.12%) ROI 152.99% Ending Value 252989
(UP DAY MA PERIOD 22) (BUY THRESH 0.60) (SELL THRESH 0.44) (MEAN TRADE PNL: -0.12%) ROI 152.99% Ending Value 252989
(UP DAY MA PERIOD 22) (BUY THRESH 0.60) (SELL THRESH 0.45) (MEAN TRADE PNL: -0.12%) ROI 152.99% Ending Value 252989
(UP DAY MA PERIOD 22) (BUY THRESH 0.60) (SELL THRESH 0.46) (MEAN TRADE PNL: -3.79%) ROI 25.64% Ending Value 125638
(UP DAY MA PERIOD 22) (BUY THRESH 0.60) (SELL THRESH 0.47) (MEAN TRADE PNL

...

(UP DAY MA PERIOD 47) (BUY THRESH 0.60) (SELL THRESH 0.49) (MEAN TRADE PNL: -14.95%) ROI 186.33% Ending Value 286334
(UP DAY MA PERIOD 47) (BUY THRESH 0.60) (SELL THRESH 0.50) (MEAN TRADE PNL: -14.95%) ROI 186.33% Ending Value 286334
(UP DAY MA PERIOD 48) (BUY THRESH 0.52) (SELL THRESH 0.40) (MEAN TRADE PNL: 14.42%) ROI 717.14% Ending Value 817144
(UP DAY MA PERIOD 48) (BUY THRESH 0.52) (SELL THRESH 0.41) (MEAN TRADE PNL: 14.42%) ROI 717.14% Ending Value 817144
(UP DAY MA PERIOD 48) (BUY THRESH 0.52) (SELL THRESH 0.42) (MEAN TRADE PNL: 5.53%) ROI 842.68% Ending Value 942677
(UP DAY MA PERIOD 48) (BUY THRESH 0.52) (SELL THRESH 0.43) (MEAN TRADE PNL: 5.53%) ROI 842.68% Ending Value 942677
(UP DAY MA PERIOD 48) (BUY THRESH 0.52) (SELL THRESH 0.44) (MEAN TRADE PNL: -4.62%) ROI 525.05% Ending Value 625046
(UP DAY MA PERIOD 48) (BUY THRESH 0.52) (SELL THRESH 0.45) (MEAN TRADE PNL: -4.62%) ROI 525.05% Ending Value 625046
(UP DAY MA PERIOD 48) (BUY THRESH 0.52) (SELL THRESH 0.46) (MEAN TRADE P

(UP DAY MA PERIOD 48) (BUY THRESH 0.58) (SELL THRESH 0.43) (MEAN TRADE PNL: 1.10%) ROI 822.34% Ending Value 922344
(UP DAY MA PERIOD 48) (BUY THRESH 0.58) (SELL THRESH 0.44) (MEAN TRADE PNL: -9.53%) ROI 649.23% Ending Value 749228
(UP DAY MA PERIOD 48) (BUY THRESH 0.58) (SELL THRESH 0.45) (MEAN TRADE PNL: -9.53%) ROI 649.23% Ending Value 749228
(UP DAY MA PERIOD 48) (BUY THRESH 0.58) (SELL THRESH 0.46) (MEAN TRADE PNL: -20.85%) ROI 412.76% Ending Value 512757
(UP DAY MA PERIOD 48) (BUY THRESH 0.58) (SELL THRESH 0.47) (MEAN TRADE PNL: -20.85%) ROI 412.76% Ending Value 512757
(UP DAY MA PERIOD 48) (BUY THRESH 0.58) (SELL THRESH 0.48) (MEAN TRADE PNL: -9.08%) ROI 192.04% Ending Value 292042
(UP DAY MA PERIOD 48) (BUY THRESH 0.58) (SELL THRESH 0.49) (MEAN TRADE PNL: -9.08%) ROI 192.04% Ending Value 292042
(UP DAY MA PERIOD 48) (BUY THRESH 0.58) (SELL THRESH 0.50) (MEAN TRADE PNL: -8.07%) ROI 153.05% Ending Value 253054
(UP DAY MA PERIOD 48) (BUY THRESH 0.59) (SELL THRESH 0.40) (MEAN TRADE 

(UP DAY MA PERIOD 49) (BUY THRESH 0.55) (SELL THRESH 0.48) (MEAN TRADE PNL: -15.58%) ROI 381.91% Ending Value 481908
(UP DAY MA PERIOD 49) (BUY THRESH 0.55) (SELL THRESH 0.49) (MEAN TRADE PNL: -18.40%) ROI 185.44% Ending Value 285436
(UP DAY MA PERIOD 49) (BUY THRESH 0.55) (SELL THRESH 0.50) (MEAN TRADE PNL: -18.40%) ROI 185.44% Ending Value 285436
(UP DAY MA PERIOD 49) (BUY THRESH 0.56) (SELL THRESH 0.40) (MEAN TRADE PNL: 13.86%) ROI 700.07% Ending Value 800072
(UP DAY MA PERIOD 49) (BUY THRESH 0.56) (SELL THRESH 0.41) (MEAN TRADE PNL: 3.25%) ROI 649.11% Ending Value 749115
(UP DAY MA PERIOD 49) (BUY THRESH 0.56) (SELL THRESH 0.42) (MEAN TRADE PNL: 3.25%) ROI 649.11% Ending Value 749115
(UP DAY MA PERIOD 49) (BUY THRESH 0.56) (SELL THRESH 0.43) (MEAN TRADE PNL: 1.09%) ROI 518.90% Ending Value 618903
(UP DAY MA PERIOD 49) (BUY THRESH 0.56) (SELL THRESH 0.44) (MEAN TRADE PNL: 1.09%) ROI 518.90% Ending Value 618903
(UP DAY MA PERIOD 49) (BUY THRESH 0.56) (SELL THRESH 0.45) (MEAN TRADE PN

(UP DAY MA PERIOD 50) (BUY THRESH 0.53) (SELL THRESH 0.42) (MEAN TRADE PNL: 4.74%) ROI 684.87% Ending Value 784872
(UP DAY MA PERIOD 50) (BUY THRESH 0.53) (SELL THRESH 0.43) (MEAN TRADE PNL: 4.74%) ROI 684.87% Ending Value 784872
(UP DAY MA PERIOD 50) (BUY THRESH 0.53) (SELL THRESH 0.44) (MEAN TRADE PNL: -7.06%) ROI 683.83% Ending Value 783835
(UP DAY MA PERIOD 50) (BUY THRESH 0.53) (SELL THRESH 0.45) (MEAN TRADE PNL: -7.06%) ROI 683.83% Ending Value 783835
(UP DAY MA PERIOD 50) (BUY THRESH 0.53) (SELL THRESH 0.46) (MEAN TRADE PNL: -19.03%) ROI 298.53% Ending Value 398531
(UP DAY MA PERIOD 50) (BUY THRESH 0.53) (SELL THRESH 0.47) (MEAN TRADE PNL: -19.03%) ROI 298.53% Ending Value 398531
(UP DAY MA PERIOD 50) (BUY THRESH 0.53) (SELL THRESH 0.48) (MEAN TRADE PNL: -11.61%) ROI 353.67% Ending Value 453670
(UP DAY MA PERIOD 50) (BUY THRESH 0.53) (SELL THRESH 0.49) (MEAN TRADE PNL: -11.61%) ROI 353.67% Ending Value 453670
(UP DAY MA PERIOD 50) (BUY THRESH 0.53) (SELL THRESH 0.50) (MEAN TRADE

(UP DAY MA PERIOD 50) (BUY THRESH 0.59) (SELL THRESH 0.47) (MEAN TRADE PNL: -22.30%) ROI 134.46% Ending Value 234464
(UP DAY MA PERIOD 50) (BUY THRESH 0.59) (SELL THRESH 0.48) (MEAN TRADE PNL: -14.73%) ROI 146.92% Ending Value 246915
(UP DAY MA PERIOD 50) (BUY THRESH 0.59) (SELL THRESH 0.49) (MEAN TRADE PNL: -14.73%) ROI 146.92% Ending Value 246915
(UP DAY MA PERIOD 50) (BUY THRESH 0.59) (SELL THRESH 0.50) (MEAN TRADE PNL: -12.29%) ROI 183.35% Ending Value 283355
(UP DAY MA PERIOD 50) (BUY THRESH 0.60) (SELL THRESH 0.40) (MEAN TRADE PNL: 16.65%) ROI 658.05% Ending Value 758054
(UP DAY MA PERIOD 50) (BUY THRESH 0.60) (SELL THRESH 0.41) (MEAN TRADE PNL: 16.65%) ROI 658.05% Ending Value 758054
(UP DAY MA PERIOD 50) (BUY THRESH 0.60) (SELL THRESH 0.42) (MEAN TRADE PNL: 2.15%) ROI 469.81% Ending Value 569806
(UP DAY MA PERIOD 50) (BUY THRESH 0.60) (SELL THRESH 0.43) (MEAN TRADE PNL: 2.15%) ROI 469.81% Ending Value 569806
(UP DAY MA PERIOD 50) (BUY THRESH 0.60) (SELL THRESH 0.44) (MEAN TRADE

In [ ]:
    # Add a strategy
#     strats = cerebro.optstrategy(
#         DIXmaopt,
#         period=range(1,26),
#         daystosell=range(30,91),
#         dixupper=range(43,50)
#     ) 

In [ ]:
%matplotlib inline
import matplotlib.pylab as pylab
pylab.rcParams['figure.figsize'] = 13, 20  # that's default image size for this interactive session
pylab.rcParams['font.family'] = 'sans-serif'
pylab.rcParams['font.sans-serif'] = ['Bitstream Vera Sans']
pylab.rcParams['font.serif'] = ['Bitstream Vera Sans']
# Plot the result
cerebro.plot()


In [ ]:
class BuyAndHold_1(bt.Strategy):
    def start(self):
        self.val_start = self.broker.get_cash()  # keep the starting cash

    def nextstart(self):
        # Buy all the available cash
        size = int(self.broker.get_cash() / self.data)
        self.buy(size=size)

    def stop(self):
        # calculate the actual returns
        #roi
        self.roi = ((self.broker.get_value()-self.val_start) / self.val_start)
        print('ROI:        {:.2f}%'.format(100.0 * self.roi))

In [ ]:
if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()    
    
    # Add a strategy
    cerebro.addstrategy(BuyAndHold_1)
    
    
    # Datas are in a subfolder of the samples. Need to find where the script is
    # because it could have been called from anywhere
    modpath = os.path.dirname(os.path.abspath(sys.argv[0]))
    datapath = os.path.join(modpath, 'C:\\Users\\big_u\\TREND WALK FORWARD TESTING\\UPDAYS2.csv')

    # Create a Data Feed
    data = custom_csv(
        dataname=datapath,
    )

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    # Add a FixedSize sizer according to the stake
    cerebro.addsizer(bt.sizers.FixedSize, stake=150)

    # Print out the starting conditions
    #print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()
    
    # Set the commission
    cerebro.broker.setcommission(commission=0.00)

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
